# Анализ БД (SQL)

**Задача:**

Проанализировать базу данных и ответить на следующие вопросы:

* Посчитать, сколько книг вышло после 1 января 2000 года
* Для каждой книги посчитать количество обзоров и среднюю оценку
* Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключим из анализа брошюры
* Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками
* Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок

**Описание данных:**

В базе данных хранятся: информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

* Таблица **books** содержит данные о книгах:
    * `book_id` — идентификатор книги
    * `author_id` — идентификатор автора
    * `title` — название книги
    * `num_pages` — количество страниц
    * `publication_date` — дата публикации книги
    * `publisher_id` — идентификатор издателя


* Таблица **authors** содержит данные об авторах:
    * `author_id` — идентификатор автора
    * `author` — имя автора
    
    
* Таблица **publishers** cодержит данные об издательствах:
    * `publisher_id` — идентификатор издательства
    * `publisher` — название издательства
    
    
* Таблица **ratings** cодержит данные о пользовательских оценках книг:
    * `rating_id` — идентификатор оценки
    * `book_id` — идентификатор книги
    * `username` — имя пользователя, оставившего оценку
    * `rating` — оценка книги
    
    
* Таблица **reviews** cодержит данные о пользовательских обзорах:
    * `review_id` — идентификатор обзора
    * `book_id` — идентификатор книги
    * `username` — имя автора обзора
    * `text` — текст обзора

## Импорт БД

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'****': '****', # имя пользователя
'pwd': '*****', # пароль
'host': '******',
'port': ****, # порт подключения
'db': '****'} # название базы данных
connection_string = '******'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'****':'****'})


## Исследование таблиц БД

In [3]:
def sql_query(table_name, limit=5):
       
    # Подготавливаем SQL-запрос с параметрами
    query = f'SELECT * FROM {table_name} LIMIT {limit}'
    
    # Подключаемся к базе данных
    con = engine.connect()
    
    # Выполняем SQL-запрос и возвращаем результат в виде DataFrame
    result = pd.io.sql.read_sql(sql=text(query), con=con)
    
    # Закрываем соединение
    con.close()
    
    return result

# Список таблиц
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

# Выполняем SQL-запросы с использованием цикла
for table in tables:
    result = sql_query(table)
    
    # Отображаем результат
    print(f"\nTable: {table}")
    display(result)



Table: books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



Table: authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



Table: publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



Table: ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



Table: reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## SQL запросы

In [4]:
def query(query):
      
    # Подключаемся к базе данных
    con = engine.connect()
    
    # Выполняем SQL-запрос и возвращаем результат в виде DataFrame
    result = pd.io.sql.read_sql(sql=text(query), con=con)
    
    # Закрываем соединение
    con.close()
    
    return result

### Задача №1

* Посчитайте, сколько книг вышло после 1 января 2000 года

In [5]:
# чтобы выполнить SQL-запрос, используем Pandas
query('''

SELECT
    COUNT(*)
FROM books
WHERE publication_date > '2000-01-01'

''')

,count
0,819


**Вывод:** После 1 января 2020 года вышло 819 книг

### Задача №2

* Для каждой книги посчитайте количество обзоров и среднюю оценку

In [6]:
# чтобы выполнить SQL-запрос, используем Pandas
query('''

SELECT
    b.book_id,
    b.title,
    COUNT(DISTINCT r.review_id) AS count_review,
    AVG(rt.rating) AS avg_rating
FROM books AS b
LEFT JOIN 
    reviews AS r ON b.book_id = r.book_id
LEFT JOIN
    ratings AS rt ON b.book_id = rt.book_id
GROUP BY
    b.book_id, b.title
ORDER BY avg_rating DESC

''')

,book_id,title,count_review,avg_rating
0,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,901,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,390,Light in August,2,5.00
3,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,316,His Excellency: George Washington,2,2.00
997,202,Drowning Ruth,3,2.00
998,371,Junky,2,2.00


**Вывод:** Были посчитаны количество обзоров (count_review) и средняя оценка(avg_rating) для 1000 книг

### Задача №3

* Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [7]:
# чтобы выполнить SQL-запрос, используем Pandas
query('''

SELECT
    p.publisher_id,
    p.publisher,
    COUNT(b.book_id) AS total_book
FROM publishers AS p
LEFT JOIN
    books AS b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY
    p.publisher_id, p.publisher
ORDER BY total_book DESC
LIMIT 1


''')

,publisher_id,publisher,total_book
0,212,Penguin Books,42


**Вывод:** Издательство **Penguin Books**  выпустило наибольшее число книг с количеством страниц выше 50. Число этих книг составило 42 шт.

### Задача №4

* Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [8]:
# чтобы выполнить SQL-запрос, используем Pandas
query('''

WITH rating_50 AS (SELECT
                        book_id
                        FROM ratings
                        GROUP BY book_id
                        HAVING COUNT(rating_id) >= 50)
SELECT
    a.author_id,
    a.author,
    AVG(r.rating) AS avg_rating
FROM authors AS a
JOIN
    books AS b ON a.author_id = b.author_id
JOIN
    ratings AS r ON b.book_id = r.book_id
JOIN
    rating_50 AS r_50 ON b.book_id = r_50.book_id
GROUP BY
    a.author_id, a.author
HAVING
    COUNT(r.rating) >= 50
ORDER BY avg_rating DESC
LIMIT 1


''')

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097


**Вывод:** Для книг от 50 оценок и выше, выявлена наибольшая средняя оценка рейтинка книг, которая составила 4,3 балла. Автором оказалась J.K. Rowling/Mary GrandPré	

### Задача №5

* Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [9]:
# чтобы выполнить SQL-запрос, используем Pandas
query('''

SELECT
    AVG(review_count) AS avg_review_count
FROM (SELECT
        username,
        COUNT(DISTINCT review_id) AS review_count
      FROM reviews
      WHERE username IN (SELECT
                             username
                         FROM ratings
                         GROUP BY username
                         HAVING COUNT(DISTINCT rating_id) > 48)
      GROUP BY username) AS user_reviews;


''')

,avg_review_count
0,24.0


**Вывод:** Для пользователей котрые поставили больше 48 оценок, посчитано среднее колличество обзоров от пользователей, которое составило 24 шт.

## ВЫВОД


1. В ходе выполнения проекта был успешно реализован импорт данных из базы данных с использованием библиотек pandas и sqlalchemy. Подключение к базе данных было настроено, и получение данных прошло успешно.

2. Исследование таблиц базы данных позволило получить первичное представление о содержании данных. Для этого был создан цикл, выполняющий SQL-запросы для каждой таблицы и выводящий результаты в виде таблиц.

3. Для оптимизации и унификации кода были использованы параметризированные запросы вместо дублирования кода для каждой таблицы. Это улучшение помогает сделать код более читаемым и поддерживаемым.

4. В разделе SQL-запросов были выполнены задачи:

   - Подсчитано количество книг, выпущенных после 1 января 2000 года.
       - **Вывод:** Результаты показывают, что после 1 января 2000 года было выпущено 819 книг. Это позволяет нам оценить объем новой литературы за указанный период.
   - Для каждой книги посчитано количество обзоров и средняя оценка.
       - **Вывод:** Запрос был успешно выполнен, и мы получили информацию о количестве обзоров и средней оценке для каждой книги. Эти данные могут быть полезны для оценки популярности и качества книг в базе данных.
   - Определено издательство, выпустившее наибольшее количество книг толще 50 страниц.
       - **Вывод:** Результаты запроса показывают, что издательство "Penguin Books" выпустило наибольшее количество книг, имеющих более 50 страниц.
   - Выявлен автор с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками.
       - **Вывод:** Мы выявили, что J.K. Rowling/Mary GrandPré является автором с самой высокой средней оценкой для книг с 50 и более оценками. Это позволяет выделить авторов, чьи произведения наиболее успешны и получают высокие оценки от читателей.
   - Посчитано среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
       - **Вывод:** Результат запроса показывает, что пользователи, поставившие более 48 оценок, в среднем написали 24 обзора. Это может быть важной информацией для понимания активности пользователей с высокой вовлеченностью, которые не только ставят оценки, но и оставляют обзоры книг.